# Imports

In [62]:
import numpy as np
import itertools as itr
import matplotlib.pyplot as plt
from time import sleep
import torch as torch
from torch import Tensor, manual_seed, is_tensor, LongTensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS, Adam
from torch.utils.data import Dataset, DataLoader

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, execute, transpile, assemble, IBMQ
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.opflow import AerPauliExpectation
from qiskit.circuit import Parameter,ParameterVector
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import EfficientSU2, RealAmplitudes, ZZFeatureMap
from qiskit.opflow.gradients import Gradient, NaturalGradient, QFI, Hessian
#from sklearn.preprocessing import OneHotEncoder
from qiskit.algorithms.optimizers import SPSA, GradientDescent, QNSPSA, ADAM
#from qiskit.tools.monitor import job_monitor


from qiskit_machine_learning.runtime import TorchRuntimeClient, TorchRuntimeResult
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import CircuitQNN, TwoLayerQNN
from qiskit_machine_learning.circuit.library import RawFeatureVector

COUNTER = 0
DIR_val_train = "Encode_data/amp_enc_data_set_trainning_values.csv"
DIR_cls_train = "Encode_data/amp_enc_data_set_trainning_classes.csv"

DIR_val_test = "Encode_data/amp_enc_data_set_test_values.csv"
DIR_cls_test = "Encode_data/amp_enc_data_set_test_classes.csv"

quantum_instance = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1024)

# Create Data Set/Loader

In [ ]:
def get_encode(file):
    """
     Funcao responsavel pelo encoding (amplitude)
        Input :: 
        ### file : File dir
        Output :: 
        #### data_enc : Valores para o encode (numpy array)
    """
    return np.genfromtxt(file, delimiter=";")

train_data = get_encode(DIR_val_train)
train_data_T = Tensor(train_data)
train_labels = np.genfromtxt(DIR_cls_train, delimiter=";")
train_labels_T = Tensor(train_labels).reshape(len(train_labels)).long()

# Test data points
test_data = get_encode(DIR_val_test)
test_data_T = Tensor(test_data)
test_labels = np.genfromtxt(DIR_cls_test, delimiter=";")
test_labels_T = Tensor(test_labels).reshape(len(test_labels)).long()

In [ ]:
# Create custom torch dataset class
class TorchDataset(Dataset):
    """Map-style dataset"""

    def __init__(self, X, y):
        self.X = Tensor(X)
        self.y = Tensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        import torch

        if torch.is_tensor(idx):
            idx = idx.tolist()

        X_i = self.X[idx]
        y_i = self.y[idx]

        # important: the dataset item must be returned as data,target
        return X_i, y_i

In [56]:
train_set = TorchDataset(train_data_T, train_labels_T)
train_loader = DataLoader(train_set, batch_size=1, shuffle=False)
test_set = TorchDataset(test_data_T, test_labels_T)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)

# Create Circuit and Torch Model

In [57]:
# Cicrcuito
n_qubits = 4

encoding = RawFeatureVector(16)
encoding_ZZ = ZZFeatureMap(feature_dimension=n_qubits, reps=2)

ansatz = EfficientSU2(n_qubits, 
                      entanglement='full', 
                      reps=4, 
                      insert_barriers=True, 
                      name="U(\u03B8)", 
                      parameter_prefix="\u03B8")

qc = QuantumCircuit(n_qubits)
qc.append(encoding, range(n_qubits))
qc.append(ansatz, range(n_qubits))
qc.decompose().draw(output="text")

parity = lambda x: "{:b}".format(x).count("1") % 2
output_shape = 2  
qnn = CircuitQNN(circuit=qc,
                 input_params=encoding.parameters,
                 weight_params=ansatz.parameters,
                 interpret=parity,
                 output_shape=output_shape,
                 sampling=True,
                 input_gradients=True, 
                 quantum_instance=quantum_instance)

initial_parameters = np.random.random(ansatz.num_parameters)

model = TorchConnector(qnn, initial_parameters)

optimizer = Adam(model.parameters(), lr=0.1)
loss_func = CrossEntropyLoss()

In sampling mode, output_shape will be automatically inferred from the number of samples and interpret function, if provided.
Cannot compute gradient operator! Continuing without gradients!


# Create Torch Frame

In [58]:
def score_func(out, target):
    from numpy import sign
    score = 0
    if sign(out.item()) == target.item():
        score = 1
    return score

In [ ]:
# Define model, optimizer, and loss
optimizer = Adam(model.parameters())
f_loss = CrossEntropyLoss()  # Our output will be in the [0,1] range

# Start training
model.train()

# Define LBFGS closure method (explained in previous section)
def closure():
    optimizer.zero_grad(set_to_none=True)  # Initialize gradient
    loss = f_loss(model(train_data_T), train_labels_T)  # Calculate loss
    loss.backward()  # Backward pass

    print(loss.item())  # Print loss
    return loss


# Run optimizer (LBFGS requires closure)
optimizer.step(closure)

In [66]:
# Example of target with class indices
loss = CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
output = loss(input, target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
print(target)
output = loss(input, target)
output.backward()

tensor([[ 0.3349,  0.3954,  0.0420, -2.1485,  0.7837],
        [-1.6283,  0.5872, -0.7256,  0.7232, -0.6321],
        [-1.1023, -1.5871, -0.9791, -2.1348,  0.6405]], requires_grad=True)
tensor([[0.1085, 0.1157, 0.0755, 0.6382, 0.0621],
        [0.1574, 0.0994, 0.4204, 0.0558, 0.2671],
        [0.0378, 0.1449, 0.2533, 0.1175, 0.4464]])


In [8]:
torch_runtime_client_TW_MSE = TorchRuntimeClient(
    provider=provider,
    model=model1,
    optimizer=optimizer1,
    loss_func=loss_func1,
    epochs=20,
    backend=backend,
)
    
fit_result_TW_MSE = torch_runtime_client_TW_MSE.fit(train_loader=train_loader)
print("train execution time: ", fit_result_TW_MSE.execution_time)

score_result = torch_runtime_client_TW_MSE.score(data_loader=test_loader, score_func=score_func)
print("id: ", score_result.job_id)
print("execution time: ", score_result.execution_time)
print("score: ", score_result.score)

train execution time:  11498.90387415886
id:  cafnv75toase3u9lgiug
execution time:  159.3931486606598
score:  0.4583333333333333


In [9]:
%%script echo skipping
fit_result_TW_MSE.model_state_dict

skipping


In [10]:
%%script echo skipping
## ERRO, mt provavelmente e o encode
fit_result_CQNN_CEL = torch_runtime_client_CQNN_CEL.fit(train_loader=train_loader, val_loader=test_loader)

skipping


In [11]:
%%script echo skipping
# In this example, we use the same data loader for the prediction as well
predict_result = torch_runtime_client_TW_MSE.predict(data_loader=train_loader)

skipping


In [12]:
print(predict_result.model_state_dict)

NameError: name 'predict_result' is not defined

In [ ]:
#%%script echo skipping
def score_func(out, target):
    from numpy import sign
    score = 0
    print("out: ", out, type(out))
    print("target: ", target)
    if sign(out.item()) == target.item():
        score = 1
    return score

In [ ]:
#%%script echo skipping
# data_loader = train_loader

score_result = torch_runtime_client_TW_MSE.score(data_loader=train_loader, score_func=score_func)

In [ ]:
%%script echo skipping
print("id: ", score_result.job_id)
print("execution time: ", score_result.execution_time)
print("score: ", score_result.score)

In [ ]:
%%script echo skipping
## data_laoder = test_loader

score_result = torch_runtime_client.score(data_loader=test_loader, score_func=score_func)

In [ ]:
%%script echo skipping
print("id: ", score_result.job_id)
print("execution time: ", score_result.execution_time)
print("score: ", score_result.score)

In [ ]:
%%script echo skipping
torch_runtime_client2 = TorchRuntimeClient(
    provider=provider,
    model=model1,
    optimizer=optimizer2,
    loss_func=loss_func2,
    epochs=50,
    backend=backend,
)

In [ ]:
%%script echo skipping
fit_result2 = torch_runtime_client2.fit(train_loader=train_loader, val_loader=test_loader)

In [ ]:
%%script echo skipping
predict_result = torch_runtime_client2.predict(data_loader=test_loader)

In [ ]:
%%script echo skipping
print(type(fit_result2.model_state_dict))

In [ ]:
%%script echo skipping
fit_result2.model_state_dict
fit_result2.val_history

In [ ]:
%%script echo skipping

In [ ]:
%%script echo skipping
print(predict_result.val_history)

In [ ]:
%%script echo skipping